In [1]:
import sys
sys.path.append('/p/home/jusers/kotobi2/juwels/hida_project/')

In [2]:
import os.path as osp
import numpy as np
import torch
from torch_geometric.loader import DataLoader

from attribution_gnn1.QM9_SpecData import QM9_SpecData
from attribution_gnn1.split import save_split

from src.models import SpectraGNN, SpectraGAT, SpectraGraphNet

from training.trainer import GNNTrainer

In [3]:
model_name = 'spectragraphnet_50k_bl.pt'
num_epochs = 100
device = 'cuda' if torch.cuda.is_available() else 'cpu'
lr =1e-3
milestones = np.arange(10, 100, 10).tolist()

# preparing the data

In [4]:
root = '/p/home/jusers/kotobi2/juwels/data_qm9/all_graph_data/qm9_spec_50k_0-8eV_bl.pt'
qm9_spec = QM9_SpecData(root=root,
             raw_dir='/p/home/jusers/kotobi2/juwels/data_qm9/raw/',
             spectra=[])#broadened_spectra_stk)

In [5]:
idxs = save_split(
    path='/p/home/jusers/kotobi2/juwels/hida_project/data/split_files2/qm9_split_50k_0-8eV_bl.npz',
    ndata=len(qm9_spec),
    ntrain=40000,
    nval=10000,
    ntest=0,
    save_split=True,
    shuffle=True, 
    print_nsample=True
)

In [6]:
#train, val and test data
train_qm9 = [qm9_spec[i] for i in idxs['train']]
val_qm9 = [qm9_spec[i] for i in idxs['val']]
#test_qm9 = qm9_spec[idxs['test']]

In [7]:
# data loaders 
train_loader = DataLoader(train_qm9, batch_size=100, shuffle=True)
val_loader = DataLoader(val_qm9, batch_size=100, shuffle=True)
#test_loader = DataLoader(test_qm9, batch_size=100)

In [17]:
len(qm9_spec)

50000

# some more metrics

In [8]:
def RSE_loss(prediction, target):
    dE = (300 - 270) / 100
    nom = torch.sum(dE*torch.pow((target-prediction), 2))
    denom = torch.sum(dE*target)
    return torch.sqrt(nom) / denom 

In [9]:
def RMSE(prediction, target):
    return torch.sqrt(torch.mean((target - prediction)**2))

# loading the model 

## spectragnn

In [17]:
spectragnn = SpectraGNN(
    gnn_name='gatv2',
    in_channels=[11, 128, 256, 512],
    out_channels=[128, 256, 512, 600],
    num_targets=100,
    num_layers=4,
    heads=3
).to(device)

# loading the saved model 
path_to_model = osp.join('./best_model', 
                         model_name)

if osp.exists(path_to_model):
    spectragnn.load_state_dict(torch.load(path_to_model))
else:
    print('model is not loaded')

model is not loaded


In [18]:
spectragnn

SpectraGNN(
  (interaction_layers): ModuleList(
    (0): GATv2Conv(11, 128, heads=3)
    (1): ReLU(inplace=True)
    (2): GATv2Conv(384, 256, heads=3)
    (3): ReLU(inplace=True)
    (4): GATv2Conv(768, 512, heads=3)
    (5): ReLU(inplace=True)
    (6): GATv2Conv(1536, 600, heads=1)
  )
  (dropout): Dropout(p=0.3, inplace=False)
  (out): Linear(in_features=600, out_features=100, bias=True)
)

## SpectraGAT

In [11]:
spectragat = SpectraGAT(
    node_features_dim=11,
    in_channels=[128, 128, 128, 128],
    out_channels=[128, 128, 128, 400],
    targets=100,
    n_layers=4,
    n_heads=3,
    gat_type = 'gatv2_custom',
    use_residuals=True,
    use_jk=True
).to(device)

# loading the saved model 
path_to_model = osp.join('./best_model', 
                         model_name)

if osp.exists(path_to_model):
    spectragat.load_state_dict(torch.load(path_to_model))
else:
    print('model is not loaded')

model is not loaded


In [12]:
spectragat

SpectraGAT(
  (pre_layer): LinearLayer(
    (linear): Linear(in_features=11, out_features=128, bias=False)
    (_activation): ReLU(inplace=True)
  )
  (res_block): Residual_block(
    (res_layers): Sequential(
      (0): LinearLayer(
        (linear): Linear(in_features=128, out_features=128, bias=False)
        (_activation): ReLU(inplace=True)
      )
      (1): LinearLayer(
        (linear): Linear(in_features=128, out_features=128, bias=False)
        (_activation): ReLU(inplace=True)
      )
      (2): LinearLayer(
        (linear): Linear(in_features=128, out_features=128, bias=False)
        (_activation): ReLU(inplace=True)
      )
      (3): LinearLayer(
        (linear): Linear(in_features=128, out_features=128, bias=False)
        (_activation): ReLU(inplace=True)
      )
    )
  )
  (gat_layers): ModuleList(
    (0): GATv2LayerCus(
      (lin_r): LinearLayer(
        (linear): Linear(in_features=128, out_features=384, bias=False)
        (_activation): Identity()
      )
  

# SpectraGraphNet

In [ ]:
spectragraphnet = SpectraGraphNet(
                 node_dim = 14,
                 edge_dim = 5,
                 hidden_channels = 512,
                 out_channels = 200,
                 gat_hidd = 512,
                 gat_out = 100,
                 n_layers = 3,
                 n_targets = 100).to(device)

# loading the saved model 
path_to_model = osp.join('./best_model', 
                         'spectragraphnet_50k_bl_pd.pt')

if osp.exists(path_to_model):
    spectragraphnet.load_state_dict(torch.load(path_to_model))
else:
    print('model is not loaded')

NameError: name 'SpectraGraphNet' is not defined

In [13]:
spectragraphnet

SpectraGraphNet(
  (graphnets): ModuleList(
    (0): GraphNetwork(
      (gatencoder): GATEncoder(
        (gats): ModuleList(
          (0): GATv2Conv(12, 64, heads=3)
          (1): ReLU(inplace=True)
          (2): GATv2Conv(192, 64, heads=3)
          (3): ReLU(inplace=True)
          (4): GATv2Conv(192, 64, heads=3)
          (5): ReLU(inplace=True)
          (6): GATv2Conv(192, 20, heads=1)
        )
      )
      (node_model): NodeModel(
        (mlp): Sequential(
          (0): Linear(in_features=42, out_features=64, bias=True)
          (1): ReLU(inplace=True)
          (2): Linear(in_features=64, out_features=50, bias=True)
          (3): ReLU(inplace=True)
          (4): LayerNorm((50,), eps=1e-05, elementwise_affine=True)
        )
      )
      (edge_model): EdgeModel(
        (mlp): Sequential(
          (0): Linear(in_features=125, out_features=64, bias=True)
          (1): ReLU(inplace=True)
          (2): Linear(in_features=64, out_features=50, bias=True)
          (3)

# Training with the trainer class

In [14]:
optimizer = torch.optim.AdamW(spectragraphnet.parameters(), lr=lr)
loss_fn = torch.nn.L1Loss()
loss_fn2 = torch.nn.MSELoss()
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, 
                                                 milestones=milestones,
                                                 gamma=0.8)

In [15]:
trainer = GNNTrainer(model=spectragraphnet, 
                     model_name="spectragraphnet_50k_bl",
                     device=device,
                     metric_path="./metrics")

In [16]:
trainer.train_val(train_loader, val_loader, optimizer,
                  RMSE, scheduler, num_epochs, write_every=1, train_graphnet=True)

  0%|          | 0/100 [00:00<?, ?it/s]/p/software/juwelsbooster/stages/2020/software/PyTorch/1.8.1-gcccoremkl-10.3.0-2021.2.0-Python-3.8.5/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:416: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
  1%|          | 1/100 [01:20<2:12:11, 80.12s/it]

time = 1.33 mins mins
epoch 0 | average train loss = 19.55  and average validation loss = 16.39  |learning rate = 0.00100


  2%|▏         | 2/100 [02:19<1:50:50, 67.86s/it]

time = 2.32 mins mins
epoch 1 | average train loss = 12.47  and average validation loss = 9.61  |learning rate = 0.00100


  3%|▎         | 3/100 [03:20<1:44:25, 64.59s/it]

...

time = 101.76 mins mins
epoch 97 | average train loss = 5.44  and average validation loss = 4.91  |learning rate = 0.00013


 99%|█████████▉| 99/100 [1:42:45<00:59, 59.78s/it]

time = 102.76 mins mins
epoch 98 | average train loss = 5.43  and average validation loss = 4.92  |learning rate = 0.00013


100%|██████████| 100/100 [1:43:44<00:00, 62.25s/it]

time = 103.75 mins mins
epoch 99 | average train loss = 5.43  and average validation loss = 4.92  |learning rate = 0.00013
